# Exercise 2.2 - 2-dim packing

In [19]:
import clingo

In [20]:
def print_answer_sets(program):
    # Load the answer set program, and call the grounder
    control = clingo.Control()
    control.add("base", [], program)
    control.ground([("base", [])])
    # Define a function that will be called when an answer set is found
    # This function sorts the answer set alphabetically, and prints it
    def on_model(model):
        sorted_model = [str(atom) for atom in model.symbols(shown=True)]
        sorted_model.sort()
        print("Answer set: {{{}}}".format(", ".join(sorted_model)))
    # Ask clingo to find all models (using an upper bound of 0 gives all models)
    control.configuration.solve.models = 0
    # Call the clingo solver, passing on the function on_model for when an answer set is found
    answer = control.solve(on_model=on_model)
    # Print a message when no answer set was found
    if answer.satisfiable == False:
        print("No answer sets")

## Example specific to exercise

In [21]:
new_program = """
    #const h=4.
    #const w=6.

    #const h1=1.
    #const w1=3.

    #const h2=2.
    #const w2=2.

    #const h3=2.
    #const w3=3.

    #const h4=3.
    #const w4=3.

    1 { corner(1,1..h-h1+1,1..w-w1+1) } 1.
    1 { corner(2,1..h-h2+1,1..w-w2+1) } 1.
    1 { corner(3,1..h-h3+1,1..w-w3+1) } 1.
    1 { corner(4,1..h-h4+1,1..w-w4+1) } 1.

    square(1,X,Y) :- corner(1,X,Y).
    square(1,X,Y+1) :- corner(1,X,Y).
    square(1,X,Y+2) :- corner(1,X,Y).

    square(2,X,Y) :- corner(2,X,Y).
    square(2,X,Y+1) :- corner(2,X,Y).
    square(2,X+1,Y) :- corner(2,X,Y).
    square(2,X+1,Y+1) :- corner(2,X,Y).

    square(3,X,Y) :- corner(3,X,Y).
    square(3,X,Y+1) :- corner(3,X,Y).
    square(3,X,Y+2) :- corner(3,X,Y).
    square(3,X+1,Y) :- corner(3,X,Y).
    square(3,X+1,Y+1) :- corner(3,X,Y).
    square(3,X+1,Y+2) :- corner(3,X,Y).

    square(4,X,Y) :- corner(4,X,Y).
    square(4,X,Y+1) :- corner(4,X,Y).
    square(4,X,Y+2) :- corner(4,X,Y).
    square(4,X+1,Y) :- corner(4,X,Y).
    square(4,X+1,Y+1) :- corner(4,X,Y).
    square(4,X+1,Y+2) :- corner(4,X,Y).
    square(4,X+2,Y) :- corner(4,X,Y).
    square(4,X+2,Y+1) :- corner(4,X,Y).
    square(4,X+2,Y+2) :- corner(4,X,Y).

    :- square(K,X,w+1).
    :- square(K,h+1,Y).
    :- square(K,X,Y), square(L,X,Y), K < L.

    #show corner/3.
    """


print_answer_sets(new_program)

Answer set: {corner(1,4,1), corner(2,1,4), corner(3,3,4), corner(4,1,1)}
Answer set: {corner(1,4,1), corner(2,1,5), corner(3,3,4), corner(4,1,1)}
Answer set: {corner(1,1,1), corner(2,1,4), corner(3,3,4), corner(4,2,1)}
Answer set: {corner(1,1,2), corner(2,1,5), corner(3,3,4), corner(4,2,1)}
Answer set: {corner(1,1,1), corner(2,1,5), corner(3,3,4), corner(4,2,1)}
Answer set: {corner(1,1,3), corner(2,1,1), corner(3,3,1), corner(4,2,4)}
Answer set: {corner(1,1,4), corner(2,1,1), corner(3,3,1), corner(4,2,4)}
Answer set: {corner(1,1,4), corner(2,1,2), corner(3,3,1), corner(4,2,4)}
Answer set: {corner(1,4,4), corner(2,1,1), corner(3,3,1), corner(4,1,4)}
Answer set: {corner(1,4,4), corner(2,1,2), corner(3,3,1), corner(4,1,4)}
Answer set: {corner(1,4,1), corner(2,3,5), corner(3,1,4), corner(4,1,1)}
Answer set: {corner(1,4,2), corner(2,3,5), corner(3,1,4), corner(4,1,1)}
Answer set: {corner(1,4,1), corner(2,3,4), corner(3,1,4), corner(4,1,1)}
Answer set: {corner(1,1,1), corner(2,3,5), corner(3

## Constructor for general problem

In [32]:
def construct_program(w, h, rectangles, show_only_corner=True):
    k = len(rectangles)
    program_str = ""
    program_str += f"\n#const h={h}."
    program_str += f"\n#const w={w}."
    program_str += "\n"

    for i, (h_, w_) in enumerate(rectangles):
        # set constants
        program_str += f"\n#const h{i+1}={h_}."
        program_str += f"\n#const w{i+1}={w_}."
        program_str += "\n"

    for i, (h_, w_) in enumerate(rectangles):
        # place corner
        program_str += f"\n 1 {{ corner({i+1},1..h-h{i+1}+1,1..w-w{i+1}+1) }} 1."
    program_str += "\n"

    for i, (h_, w_) in enumerate(rectangles):
        # fill squares from corner according to rectangle specifications
        for j in range(0, h_):
            for k in range(0, w_):
                program_str += f"\n square({i+1},X+{j},Y+{k}) :- corner({i+1},X,Y)."
        program_str += "\n"

    # constraint square overlap
    program_str += "\n:- square(K,X,Y), square(L,X,Y), K < L."

    # constraint box overflow
    program_str += "\n:- square(K,X,w+1).\n:- square(K,h+1,Y)."

    if print_answer_sets:
        program_str += "\n\n#show corner/3."

    return program_str

constructed_program = construct_program(6, 4, rectangles=[(1, 3), (2, 2), (2, 3), (3, 3)])
print(constructed_program)


#const h=4.
#const w=6.

#const h1=1.
#const w1=3.

#const h2=2.
#const w2=2.

#const h3=2.
#const w3=3.

#const h4=3.
#const w4=3.

 1 { corner(1,1..h-h1+1,1..w-w1+1) } 1.
 1 { corner(2,1..h-h2+1,1..w-w2+1) } 1.
 1 { corner(3,1..h-h3+1,1..w-w3+1) } 1.
 1 { corner(4,1..h-h4+1,1..w-w4+1) } 1.

 square(1,X+0,Y+0) :- corner(1,X,Y).
 square(1,X+0,Y+1) :- corner(1,X,Y).
 square(1,X+0,Y+2) :- corner(1,X,Y).

 square(2,X+0,Y+0) :- corner(2,X,Y).
 square(2,X+0,Y+1) :- corner(2,X,Y).
 square(2,X+1,Y+0) :- corner(2,X,Y).
 square(2,X+1,Y+1) :- corner(2,X,Y).

 square(3,X+0,Y+0) :- corner(3,X,Y).
 square(3,X+0,Y+1) :- corner(3,X,Y).
 square(3,X+0,Y+2) :- corner(3,X,Y).
 square(3,X+1,Y+0) :- corner(3,X,Y).
 square(3,X+1,Y+1) :- corner(3,X,Y).
 square(3,X+1,Y+2) :- corner(3,X,Y).

 square(4,X+0,Y+0) :- corner(4,X,Y).
 square(4,X+0,Y+1) :- corner(4,X,Y).
 square(4,X+0,Y+2) :- corner(4,X,Y).
 square(4,X+1,Y+0) :- corner(4,X,Y).
 square(4,X+1,Y+1) :- corner(4,X,Y).
 square(4,X+1,Y+2) :- corner(4,X,Y).

In [35]:
print_answer_sets(constructed_program)

Answer set: {corner(1,4,1), corner(2,1,4), corner(3,3,4), corner(4,1,1)}
Answer set: {corner(1,4,1), corner(2,1,5), corner(3,3,4), corner(4,1,1)}
Answer set: {corner(1,1,1), corner(2,1,4), corner(3,3,4), corner(4,2,1)}
Answer set: {corner(1,1,2), corner(2,1,5), corner(3,3,4), corner(4,2,1)}
Answer set: {corner(1,1,1), corner(2,1,5), corner(3,3,4), corner(4,2,1)}
Answer set: {corner(1,1,3), corner(2,1,1), corner(3,3,1), corner(4,2,4)}
Answer set: {corner(1,1,4), corner(2,1,1), corner(3,3,1), corner(4,2,4)}
Answer set: {corner(1,1,4), corner(2,1,2), corner(3,3,1), corner(4,2,4)}
Answer set: {corner(1,4,4), corner(2,1,1), corner(3,3,1), corner(4,1,4)}
Answer set: {corner(1,4,4), corner(2,1,2), corner(3,3,1), corner(4,1,4)}
Answer set: {corner(1,4,1), corner(2,3,5), corner(3,1,4), corner(4,1,1)}
Answer set: {corner(1,4,2), corner(2,3,5), corner(3,1,4), corner(4,1,1)}
Answer set: {corner(1,4,1), corner(2,3,4), corner(3,1,4), corner(4,1,1)}
Answer set: {corner(1,1,1), corner(2,3,5), corner(3

## Let's check our code with some new problems

In [59]:
# commented out are trivial cases you can use for edge case testing
problems = [
    (2, 2, [(1, 1), (1, 1), (1, 2)]),
    #(2, 2, [(3, 2)]),
    #(2, 2, [(2, 2)]),
    #(2, 2, [(1, 1), (1, 1), (2, 2)]),
    (6, 4, [(1, 3), (2, 3), (2, 3), (3, 3)]),
    (6, 4, [(1, 4), (4, 2), (3, 4)])
    ]

for problem in problems:
    print('problem: ', problem)
    print_answer_sets(construct_program(*problem))
    print("\n")

problem:  (2, 2, [(1, 1), (1, 1), (1, 2)])
Answer set: {corner(1,1,1), corner(2,1,2), corner(3,2,1)}
Answer set: {corner(1,1,2), corner(2,1,1), corner(3,2,1)}
Answer set: {corner(1,2,1), corner(2,2,2), corner(3,1,1)}
Answer set: {corner(1,2,2), corner(2,2,1), corner(3,1,1)}


problem:  (6, 4, [(1, 3), (2, 3), (2, 3), (3, 3)])
Answer set: {corner(1,4,1), corner(2,3,4), corner(3,1,4), corner(4,1,1)}
Answer set: {corner(1,1,1), corner(2,3,4), corner(3,1,4), corner(4,2,1)}
Answer set: {corner(1,4,4), corner(2,3,1), corner(3,1,1), corner(4,1,4)}
Answer set: {corner(1,1,4), corner(2,3,1), corner(3,1,1), corner(4,2,4)}
Answer set: {corner(1,4,1), corner(2,1,4), corner(3,3,4), corner(4,1,1)}
Answer set: {corner(1,1,1), corner(2,1,4), corner(3,3,4), corner(4,2,1)}
Answer set: {corner(1,4,4), corner(2,1,1), corner(3,3,1), corner(4,1,4)}
Answer set: {corner(1,1,4), corner(2,1,1), corner(3,3,1), corner(4,2,4)}


problem:  (6, 4, [(1, 4), (4, 2), (3, 4)])
Answer set: {corner(1,1,1), corner(2,1,5), 